## Noise Filter

In [12]:
import librosa
import IPython.display as ipd
import warnings
import numpy as np
import pandas as pd
import torch
from torch import nn
import soundfile as sf
warnings.filterwarnings("ignore")

In [13]:
data = pd.read_csv('Datasets/Filter_Datasets/train.csv')[:1]

In [14]:
data

,combined_path,clean_path
0,Datasets/Filter_Datasets/combined\p234_001.wav,Datasets/Filter_Datasets/clean_train\p234_001.wav


In [15]:
def audio(audio_path,label_path):
    audio, sr_audio = librosa.load(audio_path)
    label, sr_label = librosa.load(label_path)
    audio = librosa.util.normalize(audio)
    label = librosa.util.normalize(label)
    audio = librosa.stft(audio)
    label = librosa.stft(label)
    tenxor_x = torch.from_numpy(audio).T
    tenxor_x = tenxor_x.unsqueeze(0)
    tenxor_y = torch.from_numpy(label).T
    tenxor_y = tenxor_y.unsqueeze(0)
    tenxor_x = tenxor_x.float()
    tenxor_y = tenxor_y.float()
    return tenxor_x, tenxor_y

In [16]:
x_list, y_list = [], []
for x,y in zip(data["combined_path"], data["clean_path"]):
    tenxor_x, tenxor_y = audio(x,y)
    x_list.append(tenxor_x)
    y_list.append(tenxor_y)

In [103]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AudioCNN(nn.Module):
    def __init__(self):
        super(AudioCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 5), padding=(1, 2))  # Zaman ve frekans boyutlarında farklı kernel boyutları
        self.pool = nn.MaxPool2d((2, 2))
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(3, 5), padding=(1, 2))
        self.fc1 = nn.Linear(32 * 32 * 513, 1024)  # Boyutlar katmanlar ve pooling uygulamalarına bağlı olarak hesaplanmalı
        self.fc2 = nn.Linear(1024, 1025 * 125)     # Çıkış boyutunu orijinal spektrum boyutlarına uyarlayın

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 513)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.view(-1, 125, 1025)  # Çıkışı orijinal boyutlara dönüştür
        return x


In [104]:
model = AudioCNN()

In [105]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-5)

In [106]:
for e in range(1000):
    for x,y in zip(x_list,y_list):
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out,y)
        loss.backward()
        optimizer.step()
    if e % 10 == 0:
        print('Epoch:',e,'Loss:',loss.item())

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [16, 1, 3, 5], but got 3-dimensional input of size [1, 125, 1025] instead

In [80]:
from torchmetrics import SignalNoiseRatio
pred = model(x_list[-1])
snr = SignalNoiseRatio()
print(snr(pred,y_list[-1]))
pred = pred.squeeze(0).T
pred = pred.detach().numpy()

tensor(13.5518, grad_fn=<SqueezeBackward0>)


In [83]:
audio,sr = librosa.load("Datasets/Filter_Datasets/combined/p234_094.wav")
audio = librosa.util.normalize(audio)
audio = np.abs(librosa.stft(audio))
tenxor_x = torch.from_numpy(audio).T
tenxor_x = tenxor_x.unsqueeze(0)
tenxor_x = tenxor_x.float()
pred = model(tenxor_x)
pred = pred.squeeze(0).T
pred = pred.detach().numpy()

In [54]:
audio,sr = librosa.load("Datasets/Filter_Datasets/clean_train/p234_094.wav")
sf.write("clean.wav",audio,sr)

In [85]:
audio,sr = librosa.load("Datasets/Filter_Datasets/combined/p234_094.wav")
sf.write("noise.wav",audio,sr)

In [87]:
y = librosa.istft(pred)
sf.write("prediction.wav",y,sr)